
#Neuroguard, Machine learning for intracranial pressure monitoring

##Specific objectives
1. development a algoritm

##Sections of the project
1. Set up the environment

2. Download relevant databases and resources
3. Create new development space with SAM
4. Assign SAM configuration
5. Testing on random images for segmentation
6. Check out our WEB page


<a id="1.setup"></a>
#1. Set Up the environment


In [ ]:
!gdown --id 1RplOOrelIOycetRtEpoYrLmFZLtsJZUX
!7z x SAM_workshop_data2.zip

import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

import torch
import torchvision
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

!{sys.executable} -m pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!{sys.executable} -m pip install lvis
!{sys.executable} -m pip install lvis-api/.
!{sys.executable} -m pip install tqdm

from copy import deepcopy
import logging
from collections import defaultdict
from collections import OrderedDict

from lvis.lvis import LVIS
import pycocotools.mask as mask_utils
import datetime

from tqdm import tqdm
import json

from lvis import LVIS, LVISVis, LVISResults, LVISEval

In [ ]:
!wget -O kaggle.json https://raw.githubusercontent.com/aarevalom0/NeuroGuard/refs/heads/main/kaggle.json

#Some files is able to use Kaggle competitions, In this Sections we donload the files of kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Files of Kaggle competitions
!kaggle competitions download -c aptos2019-blindness-detection


--2025-01-27 13:27:16--  https://raw.githubusercontent.com/aarevalom0/NeuroGuard/refs/heads/main/kaggle.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66 [text/plain]
Saving to: ‘kaggle.json’

kaggle.json         100%[===================>]      66  --.-KB/s    in 0s      

2025-01-27 13:27:16 (1.10 MB/s) - ‘kaggle.json’ saved [66/66]

 34% 3.25G/9.51G [00:26<00:50, 134MB/s]
User cancelled operation


#2. Download relevant databases and resources